# 제9장 Tacotron 2: 일관 학습을 목표로 한 음성 합성

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch09_Tacotron.ipynb)

## 준비

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

### 패키지 임포트

In [ ]:
%pylab inline
%load_ext autoreload
%load_ext tensorboard
%autoreload
import IPython
from IPython.display import Audio
import tensorboard as tb
import os

In [ ]:
# 수치 연산
import numpy as np
import torch
from torch import nn
# 음성 파형 불러오기
from scipy.io import wavfile
# 풀 컨텍스트 라벨, 질문 파일 로드
from nnmnkwii.io import hts
# 음성 분석
import pyworld
# 음성 분석, 시각화
import librosa
import librosa.display
# 파이썬에서 배우는 음성 합성
import ttslearn

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(773)

In [ ]:
torch.__version__

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

## 9.3 엔코더

### 문자열을 숫자 열로 변환

In [ ]:
# 어휘 정의
characters = "abcdefghijklmnopqrstuvwxyz!'(),-.:;? "
# 기타 특수 기호
extra_symbols = [
    "^",  # 문장의 시작을 나타내는 특수 기호 <SOS>
    "$",  # 문장의 끝을 나타내는 특수 기호 <EOS>
]
_pad = "~"

# NOTE: 패딩을 0 번째로 배치
symbols = [_pad] + extra_symbols + list(characters)

# 문자열 ⇔ 숫자의 상호 변환을위한 사전
_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}

In [ ]:
len(symbols)

In [ ]:
def text_to_sequence(text):
    # 단순화를 위해 대문자와 소문자를 구별하지 않고 모든 대문자를 소문자로 변환
    text = text.lower()

    # <SOS>
    seq = [_symbol_to_id["^"]]

    # 본문
    seq += [_symbol_to_id[s] for s in text]

    # <EOS>
    seq.append(_symbol_to_id["$"])

    return seq


def sequence_to_text(seq):
    return [_id_to_symbol[s] for s in seq]

In [ ]:
seq = text_to_sequence("Hello!")
print(f"문자열을 숫자 열로 변환: {seq}")
print(f"숫자 열에서 문자열로 역변환: {sequence_to_text(seq)}")

### 문자 끼워 넣기

In [ ]:
class SimplestEncoder(nn.Module):
    def __init__(self, num_vocab=40, embed_dim=256):
        super().__init__()
        self.embed = nn.Embedding(num_vocab, embed_dim, padding_idx=0)
    
    def forward(self, seqs):
        return self.embed(seqs)

In [ ]:
SimplestEncoder()

In [ ]:
from ttslearn.util import pad_1d

def get_dummy_input():
    # 배치 사이즈에 2를 상정하여 적당한 문자열을 생성
    seqs = [
        text_to_sequence("What is your favorite language?"),
        text_to_sequence("Hello world."),
    ]
    in_lens = torch.tensor([len(x) for x in seqs], dtype=torch.long)
    max_len = max(len(x) for x in seqs)
    seqs = torch.stack([torch.from_numpy(pad_1d(seq, max_len)) for seq in seqs])
    
    return seqs, in_lens

In [ ]:
seqs, in_lens = get_dummy_input()
print("입력", seqs)
print("계열 길이:", in_lens)

In [ ]:
encoder = SimplestEncoder(num_vocab=40, embed_dim=256)
seqs, in_lens = get_dummy_input()
encoder_outs = encoder(seqs)
print(f"입력 사이즈: {tuple(seqs.shape)}")
print(f"출력 사이즈: {tuple(encoder_outs.shape)}")

In [ ]:
# 패딩 부분은 0을 취하고 그 이외는 연속 값으로 표현됩니다.
encoder_outs

### 1차원 컨벌루션 도입

In [ ]:
class ConvEncoder(nn.Module):
    def __init__(
        self,
        num_vocab=40,
        embed_dim=256,
        conv_layers=3,
        conv_channels=256,
        conv_kernel_size=5,
    ):
        super().__init__()
        # 문자 끼워넣기
        self.embed = nn.Embedding(num_vocab, embed_dim, padding_idx=0)

        # 1차원 컨벌루션의 중첩: 국소 의존성 모델링
        self.convs = nn.ModuleList()
        for layer in range(conv_layers):
            in_channels = embed_dim if layer == 0 else conv_channels
            self.convs += [
                nn.Conv1d(
                    in_channels,
                    conv_channels,
                    conv_kernel_size,
                    padding=(conv_kernel_size - 1) // 2,
                    bias=False,
                ),
                nn.BatchNorm1d(conv_channels),
                nn.ReLU(),
                nn.Dropout(0.5),
            ]
        self.convs = nn.Sequential(*self.convs)

    def forward(self, seqs):
        emb = self.embed(seqs)
        # 1차원 컨벌루션과 embedding 에서는 입력의 크기가 다르므로 주의
        out = self.convs(emb.transpose(1, 2)).transpose(1, 2)
        return out

In [ ]:
ConvEncoder()

In [ ]:
encoder = ConvEncoder(num_vocab=40, embed_dim=256)
seqs, in_lens = get_dummy_input()
encoder_outs = encoder(seqs)
print(f"입력 사이즈: {tuple(seqs.shape)}")
print(f"출력 사이즈: {tuple(encoder_outs.shape)}")

### 양방향 LSTM 도입

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(ConvEncoder):
    def __init__(
        self,
        num_vocab=40,
        embed_dim=512,
        hidden_dim=512,
        conv_layers=3,
        conv_channels=512,
        conv_kernel_size=5,
    ):
        super().__init__(
            num_vocab, embed_dim, conv_layers, conv_channels, conv_kernel_size
        )
        # 양방향 LSTM을 통한 장기 종속성 모델링
        self.blstm = nn.LSTM(
            conv_channels, hidden_dim // 2, 1, batch_first=True, bidirectional=True
        )

    def forward(self, seqs, in_lens):
        emb = self.embed(seqs)
        # 1차원 컨벌루션과 embedding에서는 입력의 크기가 다르므로 주의
        out = self.convs(emb.transpose(1, 2)).transpose(1, 2)

        # 양방향 LSTM 계산
        out = pack_padded_sequence(out, in_lens, batch_first=True)
        out, _ = self.blstm(out)
        out, _ = pad_packed_sequence(out, batch_first=True)
        return out

In [ ]:
Encoder()

In [ ]:
encoder = Encoder(num_vocab=40, embed_dim=256)
seqs, in_lens = get_dummy_input()
in_lens, indices = torch.sort(in_lens, dim=0, descending=True)
seqs = seqs[indices]

encoder_outs = encoder(seqs, in_lens)
print(f"입력 사이즈: {tuple(seqs.shape)}")
print(f"출력 사이즈: {tuple(encoder_outs.shape)}")

## 9.4 주의 기구

### 내용 의존주의 메카니즘

In [ ]:
from torch.nn import functional as F

# 책의 수식에 따라 이해하기 쉽도록 강조한 구현
class BahdanauAttention(nn.Module):
    def __init__(self, encoder_dim=512, decoder_dim=1024, hidden_dim=128):
        super().__init__()
        self.V = nn.Linear(encoder_dim, hidden_dim)
        self.W = nn.Linear(decoder_dim, hidden_dim, bias=False)
        # NOTE: 이 문서의 수식대로 구현하면 bias = False이지만 실용상 bias = True로 문제가 없습니다.
        self.w = nn.Linear(hidden_dim, 1)

    def forward(self, encoder_out, decoder_state, mask=None):
        # 식 (9.11) 계산
        erg = self.w(
            torch.tanh(self.W(decoder_state).unsqueeze(1) + self.V(encoder_outs))
        ).squeeze(-1)

        if mask is not None:
            erg.masked_fill_(mask, -float("inf"))

        attention_weights = F.softmax(erg, dim=1)

        # 엔코더 출력의 길이 방향에 대해 가중치 합을 취합니다.
        attention_context = torch.sum(
            encoder_outs * attention_weights.unsqueeze(-1), dim=1
        )

        return attention_context, attention_weights

In [ ]:
BahdanauAttention()

In [ ]:
from ttslearn.util import make_pad_mask

mask =  make_pad_mask(in_lens).to(encoder_outs.device)
attention = BahdanauAttention()

decoder_input = torch.ones(len(seqs), 1024)

attention_context, attention_weights = attention(encoder_outs, decoder_input, mask)

print(f"엔코더 출력 크기: {tuple(encoder_outs.shape)}")
print(f"디코더의 숨겨진 상태의 크기: {tuple(decoder_input.shape)}")
print(f"컨텍스트 벡터의 크기: {tuple(attention_context.shape)}")
print(f"어텐션 가중치: {tuple(attention_weights.shape)}")

### 하이브리드 주의 기구

In [ ]:
class LocationSensitiveAttention(nn.Module):
    def __init__(
        self,
        encoder_dim=512,
        decoder_dim=1024,
        hidden_dim=128,
        conv_channels=32,
        conv_kernel_size=31,
    ):
        super().__init__()
        self.V = nn.Linear(encoder_dim, hidden_dim)
        self.W = nn.Linear(decoder_dim, hidden_dim, bias=False)
        self.U = nn.Linear(conv_channels, hidden_dim, bias=False)
        self.F = nn.Conv1d(
            1,
            conv_channels,
            conv_kernel_size,
            padding=(conv_kernel_size - 1) // 2,
            bias=False,
        )
        # NOTE: 이 문서의 수식대로 구현하면 bias = False이지만 실용상 bias = True로 문제가 없습니다.
        self.w = nn.Linear(hidden_dim, 1)

    def forward(self, encoder_outs, src_lens, decoder_state, att_prev, mask=None):
        # 어텐션 가중치를 균일 분포로 초기화
        if att_prev is None:
            att_prev = 1.0 - make_pad_mask(src_lens).to(
                device=decoder_state.device, dtype=decoder_state.dtype
            )
            att_prev = att_prev / src_lens.unsqueeze(-1).to(encoder_outs.device)

        # (B x T_enc) -> (B x 1 x T_enc) -> (B x conv_channels x T_enc) ->
        # (B x T_enc x conv_channels)
        f = self.F(att_prev.unsqueeze(1)).transpose(1, 2)

        # 식 (9.13) 계산
        erg = self.w(
            torch.tanh(
                self.W(decoder_state).unsqueeze(1) + self.V(encoder_outs) + self.U(f)
            )
        ).squeeze(-1)

        if mask is not None:
            erg.masked_fill_(mask, -float("inf"))

        attention_weights = F.softmax(erg, dim=1)

        # 엔코더 출력의 길이 방향에 대해 가중치 합을 취합니다.
        attention_context = torch.sum(
            encoder_outs * attention_weights.unsqueeze(-1), dim=1
        )

        return attention_context, attention_weights

In [ ]:
LocationSensitiveAttention()

In [ ]:
from ttslearn.util import make_pad_mask

mask =  make_pad_mask(in_lens).to(encoder_outs.device)
attention = LocationSensitiveAttention()

decoder_input = torch.ones(len(seqs), 1024)

attention_context, attention_weights = attention(encoder_outs, in_lens, decoder_input, None, mask)

print(f"엔코더 출력 크기: {tuple(encoder_outs.shape)}")
print(f"디코더의 숨겨진 상태의 크기: {tuple(decoder_input.shape)}")
print(f"컨텍스트 벡터의 크기: {tuple(attention_context.shape)}")
print(f"어텐션 가중치: {tuple(attention_weights.shape)}")

## 9.5 디코더

### Pre-Net

In [ ]:
class Prenet(nn.Module):
    def __init__(self, in_dim, layers=2, hidden_dim=256, dropout=0.5):
        super().__init__()
        self.dropout = dropout
        prenet = nn.ModuleList()
        for layer in range(layers):
            prenet += [
                nn.Linear(in_dim if layer == 0 else hidden_dim, hidden_dim),
                nn.ReLU(),
            ]
        self.prenet = nn.Sequential(*prenet)

    def forward(self, x):
        for layer in self.prenet:
            # 학습 및 추론 모두에 Dropout을 적용합니다.
            x = F.dropout(layer(x), self.dropout, training=True)
        return x

In [ ]:
Prenet(80)

In [ ]:
decoder_input = torch.ones(len(seqs), 80)

prenet = Prenet(80)
out = prenet(decoder_input)
print(f"디코더 입력 크기: {tuple(decoder_input.shape)}")
print(f"Pre-Net 출력 크기: {tuple(out.shape)}")

### 주의 메커니즘이 있는 디코더

In [ ]:
from ttslearn.tacotron.decoder import ZoneOutCell

class Decoder(nn.Module):
    def __init__(
        self,
        encoder_hidden_dim=512,
        out_dim=80,
        layers=2,
        hidden_dim=1024,
        prenet_layers=2,
        prenet_hidden_dim=256,
        prenet_dropout=0.5,
        zoneout=0.1,
        reduction_factor=1,
        attention_hidden_dim=128,
        attention_conv_channels=32,
        attention_conv_kernel_size=31,
    ):
        super().__init__()
        self.out_dim = out_dim

        # 주의 기구
        self.attention = LocationSensitiveAttention(
            encoder_hidden_dim,
            hidden_dim,
            attention_hidden_dim,
            attention_conv_channels,
            attention_conv_kernel_size,
        )
        self.reduction_factor = reduction_factor

        # Prenet
        self.prenet = Prenet(out_dim, prenet_layers, prenet_hidden_dim, prenet_dropout)

        # 단방향 LSTM
        self.lstm = nn.ModuleList()
        for layer in range(layers):
            lstm = nn.LSTMCell(
                encoder_hidden_dim + prenet_hidden_dim if layer == 0 else hidden_dim,
                hidden_dim,
            )
            lstm = ZoneOutCell(lstm, zoneout)
            self.lstm += [lstm]

        # 출력에 projection 레이어
        proj_in_dim = encoder_hidden_dim + hidden_dim
        self.feat_out = nn.Linear(proj_in_dim, out_dim * reduction_factor, bias=False)
        self.prob_out = nn.Linear(proj_in_dim, reduction_factor)

    def _zero_state(self, hs):
        init_hs = hs.new_zeros(hs.size(0), self.lstm[0].hidden_size)
        return init_hs

    def forward(self, encoder_outs, in_lens, decoder_targets=None):
        is_inference = decoder_targets is None

        # Reduction factor를 기반으로 프레임 수 조정
        # (B, Lmax, out_dim) ->  (B, Lmax/r, out_dim)
        if self.reduction_factor > 1 and not is_inference:
            decoder_targets = decoder_targets[
                :, self.reduction_factor - 1 :: self.reduction_factor
            ]

        # 디코더의 계열 길이를 유지
        # 추론시는 엔코더의 계열 길이로부터 경험적으로 상한을 정한다
        if is_inference:
            max_decoder_time_steps = int(encoder_outs.shape[1] * 10.0)
        else:
            max_decoder_time_steps = decoder_targets.shape[1]

        # 제로 패딩된 부분에 대한 마스크
        mask = make_pad_mask(in_lens).to(encoder_outs.device)

        # LSTM 상태를 0으로 초기화
        h_list, c_list = [], []
        for _ in range(len(self.lstm)):
            h_list.append(self._zero_state(encoder_outs))
            c_list.append(self._zero_state(encoder_outs))

        # 디코더의 첫 번째 입력
        go_frame = encoder_outs.new_zeros(encoder_outs.size(0), self.out_dim)
        prev_out = go_frame

        # 이전 시간의 어텐션 가중치
        prev_att_w = None

        # 메인 루프
        outs, logits, att_ws = [], [], []
        t = 0
        while True:
            # 컨텍스트 벡터, 주의 가중치 계산
            att_c, att_w = self.attention(
                encoder_outs, in_lens, h_list[0], prev_att_w, mask
            )

            # Pre-Net
            prenet_out = self.prenet(prev_out)

            # LSTM
            xs = torch.cat([att_c, prenet_out], dim=1)
            h_list[0], c_list[0] = self.lstm[0](xs, (h_list[0], c_list[0]))
            for i in range(1, len(self.lstm)):
                h_list[i], c_list[i] = self.lstm[i](
                    h_list[i - 1], (h_list[i], c_list[i])
                )
            # 출력 계산
            hcs = torch.cat([h_list[-1], att_c], dim=1)
            outs.append(self.feat_out(hcs).view(encoder_outs.size(0), self.out_dim, -1))
            logits.append(self.prob_out(hcs))
            att_ws.append(att_w)

            # 다음 시간 디코더 입력 업데이트
            if is_inference:
                prev_out = outs[-1][:, :, -1]  # (1, out_dim)
            else:
                # Teacher forcing
                prev_out = decoder_targets[:, t, :]

            # 누적 어텐션 가중치
            prev_att_w = att_w if prev_att_w is None else prev_att_w + att_w

            t += 1
            # 정지 조건 확인
            if t >= max_decoder_time_steps:
                break
            if is_inference and (torch.sigmoid(logits[-1]) >= 0.5).any():
                break
                
        # 각 시간의 출력 결합
        logits = torch.cat(logits, dim=1)  # (B, Lmax)
        outs = torch.cat(outs, dim=2)  # (B, out_dim, Lmax)
        att_ws = torch.stack(att_ws, dim=1)  # (B, Lmax, Tmax)

        if self.reduction_factor > 1:
            outs = outs.view(outs.size(0), self.out_dim, -1)  # (B, out_dim, Lmax)

        return outs, logits, att_ws

In [ ]:
Decoder()

In [ ]:
decoder_targets = torch.ones(encoder_outs.shape[0], 120, 80)
decoder = Decoder(encoder_outs.shape[-1], 80)

# Teaccher forcing: decoder_targets(교사 데이터) 제공
with torch.no_grad():
    outs, logits, att_ws = decoder(encoder_outs, in_lens, decoder_targets);

print(f"디코더 입력 크기: {tuple(decoder_input.shape)}")
print(f"디코더 출력 크기: {tuple(outs.shape)}")
print(f"stop token (logits) 크기: {tuple(logits.shape)}")
print(f"어텐션 가중치: {tuple(att_ws.shape)}")

In [ ]:
# 자기 회귀에 기초한 추론
with torch.no_grad():
    decoder(encoder_outs[0], torch.tensor([in_lens[0]]))

## 9.6 Post-Net

In [ ]:
class Postnet(nn.Module):
    def __init__(
        self,
        in_dim=80,
        layers=5,
        channels=512,
        kernel_size=5,
        dropout=0.5,
    ):
        super().__init__()
        postnet = nn.ModuleList()
        for layer in range(layers):
            in_channels = in_dim if layer == 0 else channels
            out_channels = in_dim if layer == layers - 1 else channels
            postnet += [
                nn.Conv1d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride=1,
                    padding=(kernel_size - 1) // 2,
                    bias=False,
                ),
                nn.BatchNorm1d(out_channels),
            ]
            if layer != layers - 1:
                postnet += [nn.Tanh()]
            postnet += [nn.Dropout(dropout)]
        self.postnet = nn.Sequential(*postnet)

    def forward(self, xs):
        return self.postnet(xs)

In [ ]:
Postnet()

In [ ]:
postnet = Postnet(80)
residual = postnet(outs)

print(f"입력 크기: {tuple(outs.shape)}")
print(f"출력 크기: {tuple(residual.shape)}")

## 9.7 Tacotron 2 구현

### Tacotron 2의 모델 정의

In [ ]:
class Tacotron2(nn.Module):
    def __init__(self
    ):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.postnet = Postnet()

    def forward(self, seq, in_lens, decoder_targets):
        # 인코더로 텍스트에 잠재적인 표현 획득
        encoder_outs = self.encoder(seq, in_lens)

        # 디코더에 의한 멜 스펙트로그램, 정지 토큰 예측
        outs, logits, att_ws = self.decoder(encoder_outs, in_lens, decoder_targets)

        # Post-Net에 의한 Mel spectrogram의 잔차 예측
        outs_fine = outs + self.postnet(outs)

        # (B, C, T) -> (B, T, C)
        outs = outs.transpose(2, 1)
        outs_fine = outs_fine.transpose(2, 1)

        return outs, outs_fine, logits, att_ws
    
    def inference(self, seq):
        seq = seq.unsqueeze(0) if len(seq.shape) == 1 else seq
        in_lens = torch.tensor([seq.shape[-1]], dtype=torch.long, device=seq.device)

        return self.forward(seq, in_lens, None)

In [ ]:
seqs, in_lens = get_dummy_input()
model = Tacotron2()

# Tacotron 2 계산
outs, outs_fine, logits, att_ws = model(seqs, in_lens, decoder_targets)

print(f"입력 크기: {tuple(seqs.shape)}")
print(f"디코더 출력 크기: {tuple(outs.shape)}")
print(f"Post-Net 출력 크기: {tuple(outs_fine.shape)}")
print(f"stop token (logits) 크기: {tuple(logits.shape)}")
print(f"어텐션 가중치: {tuple(att_ws.shape)}")

In [ ]:
model

### 장난감 모델을 이용한 Tacotron 2의 동작 확인

In [ ]:
from ttslearn.tacotron import Tacotron2
model = Tacotron2(encoder_conv_layers=1, decoder_prenet_layers=1, decoder_layers=1, postnet_layers=1)

In [ ]:
def get_dummy_inout():
    seqs, in_lens = get_dummy_input()
   
    # 디코더 출력(멜 스펙트로그램)의 교사 데이터
    decoder_targets = torch.ones(2, 120, 80)
    
    # stop token 교사 데이터
    # stop token의 예상 값은 확률이지만 교사 데이터는 이진 레이블입니다.
    # 1은 디코더 출력이 완료되었음을 나타냅니다.
    stop_tokens = torch.zeros(2, 120)
    stop_tokens[:, -1:] = 1.0
    
    return seqs, in_lens, decoder_targets, stop_tokens

In [ ]:
# 적절한 입출력 생성
seqs, in_lens, decoder_targets, stop_tokens = get_dummy_inout()

# Tacotron 2의 출력 계산
# NOTE: teacher-forcing 를 위해, decoder targets 를 명시적으로 준다
outs, outs_fine, logits, att_ws = model(seqs, in_lens, decoder_targets)

print("입력 크기:", tuple(seqs.shape))
print("디코더 출력 크기:", tuple(outs.shape))
print("Stop token 크기:", tuple(logits.shape))
print("어텐션 가중치:", tuple(att_ws.shape))

### Tacotron 2の損失関数の計算

In [ ]:
# 1. 디코더 출력에 대한 손실
out_loss = nn.MSELoss()(outs, decoder_targets)
# 2. Post-Net 이후의 출력에 대한 손실
out_fine_loss = nn.MSELoss()(outs_fine, decoder_targets)
# 3. Stop token에 대한 손실
stop_token_loss = nn.BCEWithLogitsLoss()(logits, stop_tokens)

In [ ]:
print("out_loss: ", out_loss.item())
print("out_fine_loss: ", out_fine_loss.item())
print("stop_token_loss: ", stop_token_loss.item())